## Finding correlation in playType's id and description

In [ ]:
import io, json, os

In [ ]:
files = [x for x in os.listdir() if 'json' in x]
#playType = dict()

In [ ]:
for file in files:
    with io.open(file , 'r') as jsonfile:
        data = json.load(jsonfile)
        for item in data['commentary']['items']:
            #print (item['playType']['id'], item['playType']['description'])
            if item['playType']['id'] not in playType:
                playType[item['playType']['id']] = item['playType']['description']

In [ ]:
playType

## testing and looking for patterns

In [ ]:
t_commentary = 'commentary'
t_items = 'items'
t_shorttext = 'shortText'
t_postText = 'postText'
t_text = 'text'
t_preText = 'preText'

In [ ]:
for filename in files:
    with open(filename, 'r') as file:
        data = json.load(file)

    for item in data[t_commentary][t_items]:
#         print (t_postText, item[t_postText])
        print (filename)
        print(t_preText, item[t_preText])
#         print (t_text, item[t_text])
#         print ('\n', t_shorttext, item[t_shorttext])

#### *looks like postText contains valueable data on a person for analysis* I'll have to detect the named entity of a person and add to his score from any sentence that has his name

preText is valuable but only **if it has names**

# Let's create a function for person detection

In [1]:
from bs4 import BeautifulSoup as bsp
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

def without_stopwords(word_list):
    stopset = set(stopwords.words('english'))
    return [x for x in word_list if x not in stopset]

text = 'Soumya and Tamim are headed out to the middle as are the South African team. Tamim will face the first ball. His wrist must be OK. Lungi Ngidi has the new ball. Here we go.\n\n<strong>10.22am</strong> The teams are out on the ground for the anthems.\n\nNews from the South African camp regarding Hashim Amla\'s health. Proteas team manager, Dr Mohammed Moosajee explained:  "Hashim hasn\'t fully recovered from the blow sustained to the helmet during the match against England on Thursday and is not available for selection for the match today. With the quick turnaround between the two matches, he is being rested as a precaution, based on our post-match re-assessment. We are hopeful that he will make a full recovery to be available for selection for our next match against India.'
sentence = bsp(text, 'lxml').text

In [2]:
sentence

'Soumya and Tamim are headed out to the middle as are the South African team. Tamim will face the first ball. His wrist must be OK. Lungi Ngidi has the new ball. Here we go.\n\n10.22am The teams are out on the ground for the anthems.\n\nNews from the South African camp regarding Hashim Amla\'s health. Proteas team manager, Dr Mohammed Moosajee explained:  "Hashim hasn\'t fully recovered from the blow sustained to the helmet during the match against England on Thursday and is not available for selection for the match today. With the quick turnaround between the two matches, he is being rested as a precaution, based on our post-match re-assessment. We are hopeful that he will make a full recovery to be available for selection for our next match against India.'

In [3]:
import requests, io, json
r = requests.get('http://www.espncricinfo.com/series/engine/match/1144487.json')
match_summ = r.json()

In [4]:
player_names = dict()
for item in match_summ['team'][0]['player']:
    player_names[item['known_as'].lower()] = list([item['card_long'].lower(),item['popular_name'].lower(),item['known_as'].lower()])
#     name_list.extend([item['card_long'],item['card_short'],item['known_as']])
#     name_list.add(item['card_long'])
#     name_list.add(item['popular_name'])
#     name_list.add(item['known_as'])
# player_names = set(name_list)

In [5]:
'tamim' in player_names['tamim iqbal']

True

In [6]:
with io.open('player_names.json', 'w') as fp:
    json.dump(player_names, fp)

In [54]:
for key in player_names.keys():
    if 'shakib' in player_names[key]:
        print ('ok')
#     print(player_names[key])

In [21]:
personal_comm = dict()

In [22]:
import nltk
with io.open('player_names.json', 'r') as fp:
    players = json.load(fp)
for sent in nltk.sent_tokenize(sentence):
    print (sent)
    for chunk in nltk.ne_chunk(nltk.pos_tag(without_stopwords(nltk.word_tokenize(sent)))):
        if hasattr(chunk, 'label') and (chunk.label() == 'PERSON'):
            playerFromSent = ' '.join(c[0] for c in chunk)
#             print(playerFromSent)
            for player in players.keys():
                if playerFromSent.lower() in players[player]:
                    print (playerFromSent,'found')
                    personal_comm.setdefault(player.lower(),[]).append((0,sent))  

Soumya and Tamim are headed out to the middle as are the South African team.
Soumya found
Tamim found
Tamim will face the first ball.
His wrist must be OK. Lungi Ngidi has the new ball.
Here we go.
10.22am The teams are out on the ground for the anthems.
News from the South African camp regarding Hashim Amla's health.
Proteas team manager, Dr Mohammed Moosajee explained:  "Hashim hasn't fully recovered from the blow sustained to the helmet during the match against England on Thursday and is not available for selection for the match today.
With the quick turnaround between the two matches, he is being rested as a precaution, based on our post-match re-assessment.
We are hopeful that he will make a full recovery to be available for selection for our next match against India.


In [26]:
personal_comm

{'soumya sarkar': [(0,
   'Soumya and Tamim are headed out to the middle as are the South African team.'),
  (5, 'useful comm')],
 'tamim iqbal': [(0,
   'Soumya and Tamim are headed out to the middle as are the South African team.')]}

In [27]:
del personal_comm['soumya sarkar']['lalue']

TypeError: list indices must be integers or slices, not str

In [25]:
personal_comm.setdefault('soumya sarkar',[]).append((5,"useful comm"))